In [1]:
import os
import numpy as np
import aocd
import math
from aocd.models import Puzzle
from aocd import submit
%pdb

Automatic pdb calling has been turned ON


In [2]:
current_day = 16
current_year = 2021
puzzle = Puzzle(year=current_year, day=current_day)
puzzle

<Puzzle(2021, 16) at 0x7fd1c440fbe0 - Packet Decoder>

# Part 1

In [3]:
data = puzzle.input_data.strip()
data

'20546718027401204FE775D747A5AD3C3CCEEB24CC01CA4DFF2593378D645708A56D5BD704CC0110C469BEF2A4929689D1006AF600AC942B0BA0C942B0BA24F9DA8023377E5AC7535084BC6A4020D4C73DB78F005A52BBEEA441255B42995A300AA59C27086618A686E71240005A8C73D4CF0AC40169C739584BE2E40157D0025533770940695FE982486C802DD9DC56F9F07580291C64AAAC402435802E00087C1E8250440010A8C705A3ACA112001AF251B2C9009A92D8EBA6006A0200F4228F50E80010D8A7052280003AD31D658A9231AA34E50FC8010694089F41000C6A73F4EDFB6C9CC3E97AF5C61A10095FE00B80021B13E3D41600042E13C6E8912D4176002BE6B060001F74AE72C7314CEAD3AB14D184DE62EB03880208893C008042C91D8F9801726CEE00BCBDDEE3F18045348F34293E09329B24568014DCADB2DD33AEF66273DA45300567ED827A00B8657B2E42FD3795ECB90BF4C1C0289D0695A6B07F30B93ACB35FBFA6C2A007A01898005CD2801A60058013968048EB010D6803DE000E1C6006B00B9CC028D8008DC401DD9006146005980168009E1801B37E02200C9B0012A998BACB2EC8E3D0FC8262C1009D00008644F8510F0401B825182380803506A12421200CB677011E00AC8C6DA2E918DB454401976802F29AA324A6A8C12B3FD978004EB30076194278BE600C

In [4]:
bitmap = {'0':'0000',
          '1':'0001',
          '2':'0010',
          '3':'0011',
          '4':'0100',
          '5':'0101',
          '6':'0110',
          '7':'0111',
          '8':'1000',
          '9':'1001',
          'A':'1010',
          'B':'1011',
          'C':'1100',
          'D':'1101',
          'E':'1110',
          'F':'1111'}

In [5]:
version_bits = 3
type_bits = 3
literal_bits = 5
len_subpackets_bits = 15
num_subpackets_bits = 11

In [6]:
def parse_literal(bitstream, index):
    literal_string = []
    while bitstream[index] == '1':
        # keep appending next 4 bits
        literal_string.append(bitstream[index+1:index+literal_bits])
        index += literal_bits
    # last one
    literal_string.append(bitstream[index+1:index+literal_bits])
    index += literal_bits
    return int(''.join(literal_string),2), index

In [7]:
def parse_bitstream(bits, idx):
    packet_version = int(bits[idx:idx+version_bits],2)
    idx += version_bits
    packet_type = int(bits[idx:idx+type_bits],2)
    idx += type_bits
    result = {'version':packet_version, 'type': packet_type}
    if packet_type == 4: # literal
        literal, idx = parse_literal(bits, idx)
        # print(f'Found literal with value = {literal}')
        result['value'] = literal
    else: # operator
        result['value'] = []
        if bits[idx] == '0':
            # next 15 bits represent total length in bits of sub-packets
            len_subpackets = int(bits[idx+1:idx+1+len_subpackets_bits],2)
            idx += 1+len_subpackets_bits
            target_idx = idx + len_subpackets
            # print(f"Found operator with {len_subpackets} bits' worth of subpackets...")
            while idx < target_idx:
                res, idx = parse_bitstream(bits, idx)
                result['value'].append(res)
        else:
            # next 11 bits represent number of sub-packets
            num_subpackets = int(bits[idx+1:idx+1+num_subpackets_bits],2)
            idx += 1+num_subpackets_bits
            # print(f"Found operator with {num_subpackets} subpackets...")
            for subpacket in range(num_subpackets):
                res, idx = parse_bitstream(bits, idx)
                result['value'].append(res)
            
    return result, idx

In [8]:
def get_version_sum(D):
    version_sum = D['version']
    # if thiss contains other packets, recursively get sum of their version_sums
    if type(D['value']) is list:
        version_sum += sum(get_version_sum(x) for x in D['value'])
    return version_sum
    


In [9]:
def solve_part1(data):
    # construct the bitstream
    bits = ''.join(bitmap[i] for i in data)
    # parse the bitstream
    result, idx = parse_bitstream(bits, 0)
    # compute the version sum
    version_sum = get_version_sum(result)
    # return the result nested dictionary structure along with version sum
    return version_sum, result



In [10]:
version_sum, result = solve_part1('D2FE28')
print(f"{version_sum = }")
display(result)

version_sum = 6


{'version': 6, 'type': 4, 'value': 2021}

In [11]:
version_sum, result = solve_part1('EE00D40C823060')
print(f"{version_sum = }")
display(result)

version_sum = 14


{'version': 7,
 'type': 3,
 'value': [{'version': 2, 'type': 4, 'value': 1},
  {'version': 4, 'type': 4, 'value': 2},
  {'version': 1, 'type': 4, 'value': 3}]}

In [12]:
version_sum, result = solve_part1('8A004A801A8002F478')
print(f"{version_sum = }")
display(result)

version_sum = 16


{'version': 4,
 'type': 2,
 'value': [{'version': 1,
   'type': 2,
   'value': [{'version': 5,
     'type': 2,
     'value': [{'version': 6, 'type': 4, 'value': 15}]}]}]}

In [13]:
version_sum, result = solve_part1('620080001611562C8802118E34')
print(f"{version_sum = }")
display(result)

version_sum = 12


{'version': 3,
 'type': 0,
 'value': [{'version': 0,
   'type': 0,
   'value': [{'version': 0, 'type': 4, 'value': 10},
    {'version': 5, 'type': 4, 'value': 11}]},
  {'version': 1,
   'type': 0,
   'value': [{'version': 0, 'type': 4, 'value': 12},
    {'version': 3, 'type': 4, 'value': 13}]}]}

In [14]:
version_sum, result = solve_part1('C0015000016115A2E0802F182340')
print(f"{version_sum = }")
display(result)

version_sum = 23


{'version': 6,
 'type': 0,
 'value': [{'version': 0,
   'type': 0,
   'value': [{'version': 0, 'type': 4, 'value': 10},
    {'version': 6, 'type': 4, 'value': 11}]},
  {'version': 4,
   'type': 0,
   'value': [{'version': 7, 'type': 4, 'value': 12},
    {'version': 0, 'type': 4, 'value': 13}]}]}

In [15]:
version_sum, result = solve_part1('A0016C880162017C3686B18A3D4780')
print(f"{version_sum = }")
display(result)

version_sum = 31


{'version': 5,
 'type': 0,
 'value': [{'version': 1,
   'type': 0,
   'value': [{'version': 3,
     'type': 0,
     'value': [{'version': 7, 'type': 4, 'value': 6},
      {'version': 6, 'type': 4, 'value': 6},
      {'version': 5, 'type': 4, 'value': 12},
      {'version': 2, 'type': 4, 'value': 15},
      {'version': 2, 'type': 4, 'value': 15}]}]}]}

In [16]:
version_sum, result = solve_part1(puzzle.input_data.strip())
print(f"{version_sum = }")
display(result)

version_sum = 955


{'version': 1,
 'type': 0,
 'value': [{'version': 6,
   'type': 1,
   'value': [{'version': 3,
     'type': 5,
     'value': [{'version': 0, 'type': 4, 'value': 261839571},
      {'version': 6, 'type': 4, 'value': 418703}]},
    {'version': 1, 'type': 4, 'value': 55956}]},
  {'version': 6,
   'type': 3,
   'value': [{'version': 4, 'type': 4, 'value': 49045},
    {'version': 4, 'type': 4, 'value': 9901349},
    {'version': 3, 'type': 4, 'value': 4},
    {'version': 2, 'type': 4, 'value': 90535300}]},
  {'version': 6,
   'type': 3,
   'value': [{'version': 1, 'type': 4, 'value': 8},
    {'version': 6, 'type': 4, 'value': 777300},
    {'version': 4, 'type': 4, 'value': 17943}]},
  {'version': 2,
   'type': 1,
   'value': [{'version': 7,
     'type': 5,
     'value': [{'version': 5, 'type': 4, 'value': 2117752},
      {'version': 1, 'type': 4, 'value': 2117752}]},
    {'version': 4, 'type': 4, 'value': 247}]},
  {'version': 3,
   'type': 2,
   'value': [{'version': 1, 'type': 4, 'value': 1

In [17]:
puzzle.answer_a = version_sum

# Part 2

In [18]:
def process_packet(packet):
    if packet['type'] == 0:
        return sum(process_packet(x) for x in packet['value'])
    elif packet['type'] == 1:
        return math.prod(process_packet(x) for x in packet['value'])
    elif packet['type'] == 2:
        return min(process_packet(x) for x in packet['value'])
    elif packet['type'] == 3:
        return max(process_packet(x) for x in packet['value'])
    elif packet['type'] == 4:
        return packet['value']
    elif packet['type'] == 5:
        packet1, packet2 = packet['value']
        return process_packet(packet1) > process_packet(packet2)
    elif packet['type'] == 6:
        packet1, packet2 = packet['value']
        return process_packet(packet1) < process_packet(packet2)
    elif packet['type'] == 7:
        packet1, packet2 = packet['value']
        return process_packet(packet1) == process_packet(packet2)
    

In [19]:
value = process_packet(result)
value

158135423448

In [20]:
puzzle.answer_b = value